In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import gzip
import matplotlib.colors as mcol
import matplotlib.cm as cm
import pickle
import os
import imageio
import gc
import cv2
from scipy import stats
from scipy import ndimage #for finding com
from scipy.misc import imsave
from itertools import cycle
cycol = cycle('bgrcmk') #here I'm generating a list of colors I can plot

In [ ]:
with open(experiment+'_variables', 'rb') as variables:
    number_of_frames,frame_height,frame_width,fps,background_chunk = pickle.load(variables)
    
exp_numb=0
box_size=192
stringency=10
grand_frame=int(exp_numb*(number_of_frames-background_chunk))
min_range=0
max_range=800 #1000

In [ ]:
experiment=input("Regarding experiment date and number (six digit id): Enter YYMMDD_######   -")
os.mkdir(experiment)

In [ ]:
depthfilename='%s_raw_depth.gz'%experiment
colorfilename='%s_raw_color.gz'%experiment

In [ ]:
#Load and uncompress data
input_file = gzip.GzipFile(depthfilename, 'rb')
data = input_file.read()
input_file.close()

output_file = open("depthdata.bin", 'wb')
output_file.write(data)
output_file.close()

In [ ]:
background_frames = np.fromfile('depthdata.bin', dtype=np.uint16, count=(int(frame_height*frame_width*background_chunk*16))).reshape(-1, frame_height, frame_width)[0:background_chunk]

backgroundmean=np.mean(background_frames, axis=0) #may be better to use median
backgroundvariance=np.var(background_frames, axis=0)

In [ ]:
plt.imshow(background_frames[6])

In [ ]:
current_frame=0
frames = np.fromfile('depthdata.bin', dtype=np.uint16).reshape(-1, frame_height, frame_width)[background_chunk:number_of_frames_to_record]
for f in range(int(number_of_frames_to_record-background_chunk)):
        
    #FRAME TO PROCESS
    frame_of_interest=frames[current_frame] 
        
    #IF PIXEL ISN'T SIGNIFICANTLY DIFFERENT FROM THE BACKGROUND THEN DROP IT
    backgroundmultiplier=np.abs(frame_of_interest - backgroundmean) - (stringency*np.sqrt(backgroundvariance)) 

    #WE EXPECT POSITIVE VALUES FOR HIGH DIFFERENCES make them stay by multiplying by 1
    backgroundmultiplier[backgroundmultiplier>0]=1
    #WE EXPECT NEGATIVE VALUES FOR SMALL OR NO DIFFERENCE probably just noise, get rid of them by multiplying by 0
    backgroundmultiplier[backgroundmultiplier<=0]=0
       
    # SUBTRACT BACKGROUND BY MULTIPLYING INSIGNIFIACNT FRAMES BY ZERO
    frame_of_interest_backgroundsubtracted=(np.multiply(frame_of_interest,backgroundmultiplier)).astype('uint8') #IMWRITE TAKES UINT8
    frame_of_interest_backgroundsubtracted[frame_of_interest_backgroundsubtracted>max_range]=0
    frame_of_interest_backgroundsubtracted[frame_of_interest_backgroundsubtracted<=min_range]=0
        
    #################################################################
    #ALGORITHIM FOR FINDING THE CENTER OF THE OBJECT (TO CROP THE PLOT) (this actually works better than any built in function I've tried)
    #basically, the row and column with the most non zero values should be the center
        
    winningrow=0
    winningcountr=frame_width
    for rowpixels in range(0,frame_height):
        currentcountr=np.count_nonzero(frame_of_interest_backgroundsubtracted[rowpixels,:]==0)
        if currentcountr<winningcountr:
            winningcountr=currentcountr
            winningrow=rowpixels

    winningcolumn=0
    winningcountc=video_height
    for columnpixels in range(0,video_width):
        currentcountc=np.count_nonzero(frame_of_interest_backgroundsubtracted[:,columnpixels]==0)
        if currentcountc<winningcountc:
            winningcountc=currentcountc
            winningcolumn=columnpixels

    if winningrow<(box_size/2):
        winningrow=(int(box_size/2))
    if winningrow>(frame_height-box_size/2):
        winningrow=int(frame_height-box_size/2-1)
    if winningcolumn<(box_size/2):
        winningcolumn=(int(box_size/2))
    if winningcolumn>(frame_width-box_size/2):
        winningcolumn=int(frame_width-box_size/2-1)
        
        ####################################################################

        #THIS IS THE FRAME NUMBER (I WILL COMBINE MANY RUNS SO I'M OUTPUTTING THE MASTER FRAME NUMBER)
    fplusd=int(f+grand_frame)
    print(fplusd)
       

        # THIS IS HOW I SHOULD SAVE ARRAYS SO THEY COME OUT AS 192 by 192 FOR IMAGE ANALYSIS (IT'S SMALLER!)
    imageio.imwrite('%s/depth_frame_%06d.png' %(experiment,fplusd),frame_of_interest_backgroundsubtracted[winningrow-int(box_size/2):winningrow+int(box_size/2),winningcolumn-int(box_size/2):winningcolumn+int(box_size/2)])

    current_frame=current_frame+1
        
        
    